# Create dataframe of variant calls in ancestral sequence and analyze

In [1]:
#load necessary packages
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
%matplotlib inline

## Import data from ancestral sequence vcf file

In [2]:
#Define functions needed to convert vcf file into dataframe

def vcf2tsv(vcf):
    
    file = open(vcf)
    lines = file.readlines()
    file.close()
    out_file = open('temp.tsv','w')
    
    for line in lines:
        if line[0:2] != '##':
            out_file.write(line)
    
    out_file.close()
    
    

#determines total number of reads
def list_sum(in_list):
    total = 0
    for k in range(len(in_list)):
        total = total+int(in_list[k])
    return(total)

In [3]:
#upload data
vcf2tsv('/Users/chrisgraves/Documents/Yeast_data/Sequencing/alignments/vcf/merged/P2-0.vcf')
parent = pd.read_csv('temp.tsv',sep='\t')
parent.columns = ['Chrom','Pos','ID','Ref','Alt','Qual','Filter','Info','Format','Unknown']


#create dict containing chromosome IDs
chrom_key = list(parent['Chrom'].unique())
chrom_num = list(['I','II','III','IV','V','VI','VII','VIII','IX','X','XI','XII','XIII','XIV','XV','XVI','MITO'])
chrom_dict = dict(zip(chrom_key,chrom_num))

parent.head()

,Chrom,Pos,ID,Ref,Alt,Qual,Filter,Info,Format,Unknown
0,ref|NC_001133|,10,.,A,C,0.000089,.,AB=0;ABP=0;AC=0;AF=0;AN=2;AO=2;CIGAR=1X;DP=6;D...,GT:DP:RO:QR:AO:QA:GL,"0/0:6:4:140:2:4:0,-0.250089,-12.57"
1,ref|NC_001133|,6736,.,CAAAAAAAAAAAAAAAAAAAT,CAAAAAAAAAAAAAAAAAAAAAT,265.314000,.,AB=0;ABP=0;AC=2;AF=1;AN=2;AO=10;CIGAR=1M2I20M;...,GT:DP:RO:QR:AO:QA:GL,"1/1:19:0:0:10:334:-30.1609,-3.0103,0"
2,ref|NC_001133|,12594,.,A,T,720.182000,.,AB=0.201087;ABP=145.808;AC=1;AF=0.5;AN=2;AO=37...,GT:DP:RO:QR:AO:QA:GL,"0/1:184:144:4808:37:1286:-99.969,0,-416.938"
3,ref|NC_001133|,12690,.,A,T,1828.060000,.,AB=0.333333;ABP=50.7827;AC=1;AF=0.5;AN=2;AO=66...,GT:DP:RO:QR:AO:QA:GL,"0/1:198:130:4461:66:2225:-194.579,0,-395.828"
4,ref|NC_001133|,19971,.,CT,CCTT,3947.170000,.,AB=0;ABP=0;AC=2;AF=1;AN=2;AO=131;CIGAR=1M2I1M;...,GT:DP:RO:QR:AO:QA:GL,"1/1:133:0:0:131:4450:-400.554,-39.4349,0"


In [4]:
#Build dataframe of ancestral sequence
parent_df = pd.DataFrame(columns = ['Chromosome','Position','Ref','Alt','Ref_count','Alt_count','Ref_freq','Alt_freq','N_calls'])
ind=0

for index, row in parent.iterrows():
    chrom = chrom_dict[row['Chrom']] #change from ref # to chromosome # using dict
    pos = int(row['Pos'])
    ref = row['Ref']
    alt_list = row['Alt'].split(',') #split into alternate alleles
    num_alleles = len(alt_list) #determine number of alternate alleles
    merged_list = row['Format'].split(':') #determine how string of merged data is formatted
    data = row['Unknown'].split(':')
    data_dict = dict(zip(merged_list,data))
    ref_count = int(data_dict['RO'])
    alt_counts = data_dict['AO'].split(',')
    total_alt = list_sum(alt_counts)
    total_count = total_alt+ref_count
    ref_freq = float(ref_count)/float(total_count)
    for alt_ind in range(len(alt_list)):
        alt = alt_list[alt_ind]
        alt_count = alt_counts[alt_ind]
        alt_freq = float(alt_count)/float(total_count)
        parent_df.loc[ind] = (chrom,pos,ref,alt,ref_count,alt_count,ref_freq,alt_freq,total_count)
        ind = ind+1
        
parent_df.head()

,Chromosome,Position,Ref,Alt,Ref_count,Alt_count,Ref_freq,Alt_freq,N_calls
0,I,10,A,C,4,2,0.666667,0.333333,6
1,I,6736,CAAAAAAAAAAAAAAAAAAAT,CAAAAAAAAAAAAAAAAAAAAAT,0,10,0.000000,1.000000,10
2,I,12594,A,T,144,37,0.795580,0.204420,181
3,I,12690,A,T,130,66,0.663265,0.336735,196
4,I,19971,CT,CCTT,0,131,0.000000,1.000000,131


In [5]:
#output parent df to csv
parent_df.to_csv('/Users/chrisgraves/Documents/Yeast_data/Sequencing/saved_dfs/parent_df.csv',index=False)

## Check that alleles fixed in parent are also fixed in samples 

In [6]:
alleles = pd.read_csv('/Users/chrisgraves/Documents/Yeast_data/Sequencing/saved_dfs/combined_alleles.csv')
num_alleles = alleles.shape[0]/6
print('There are %d alleles' %num_alleles)
alleles.head()

There are 64499 alleles


,Unnamed: 0,Chromosome,Position,Alt,Time,Ref,Is_repeat,Read_depth,Num_ref,Num_alt,Freq_ref,Freq_alt,Qual_ref,Qual_alt,Treatment,Strain
0,0,I,5068,ATTTTTTTTTTTTTTC,1,ATTTTTTTTTTTTTC,1,NaN,NaN,0,NaN,0,NaN,NaN,C,1
1,1,I,5068,ATTTTTTTTTTTTTTC,3,ATTTTTTTTTTTTTC,1,NaN,NaN,0,NaN,0,NaN,NaN,C,1
2,2,I,5068,ATTTTTTTTTTTTTTC,5,ATTTTTTTTTTTTTC,1,NaN,NaN,0,NaN,0,NaN,NaN,C,1
3,3,I,5068,ATTTTTTTTTTTTTTC,7,ATTTTTTTTTTTTTC,1,NaN,NaN,0,NaN,0,NaN,NaN,C,1
4,4,I,5068,ATTTTTTTTTTTTTTC,9,ATTTTTTTTTTTTTC,1,NaN,NaN,0,NaN,0,NaN,NaN,C,1


In [7]:
all_alleles = alleles.groupby(['Chromosome','Position','Alt'])

In [8]:
#script to determine whether fixed alleles in ancestor are also present in experimental samples
num_fixed_P = 0
num_fixed_samples = 0
unique2P = list()

for name, group in parent_df.groupby(['Chromosome','Position','Alt']):
    if group['Alt_freq'].iloc[0] > 0.9: #if the allele is at a frequency greater than 0.9 in the ancestor
        num_fixed_P = num_fixed_P+1
        
        chrom = name[0]
        try:
            in_samples = all_alleles.get_group((name[0],int(name[1]),name[2]))
            if in_samples['Freq_alt'].sum() > 100: #frequencies should sum to 120 if fixed at all 6 timepoints in all 120 samples
                num_fixed_samples = num_fixed_samples+1             
            else:
                print('Mutation %s not fixed in all samples' %str(name))
        except KeyError:        
            unique2P.append(((name[0],int(name[1]),name[2])))
            print('Mutation %s only found in parent' %str(name))
            
print('There are %d fixed differences between the ancestor and reference. Only %d of these differences were fixed in evolved strains' %(num_fixed_P,num_fixed_samples))
        

Mutation ('I', 6736.0, 'CAAAAAAAAAAAAAAAAAAAAAT') not fixed in all samples
Mutation ('I', 25536.0, 'A') not fixed in all samples
Mutation ('I', 25542.0, 'G') not fixed in all samples
Mutation ('I', 26103.0, 'C') not fixed in all samples
Mutation ('I', 26121.0, 'A') not fixed in all samples
Mutation ('I', 27075.0, 'A') not fixed in all samples
Mutation ('I', 27080.0, 'TAGTA') not fixed in all samples
Mutation ('I', 27090.0, 'C') not fixed in all samples
Mutation ('I', 27098.0, 'AC') not fixed in all samples
Mutation ('I', 223165.0, 'T') not fixed in all samples
Mutation ('II', 259764.0, 'T') not fixed in all samples
Mutation ('II', 263153.0, 'T') not fixed in all samples
Mutation ('II', 263158.0, 'CG') not fixed in all samples
Mutation ('II', 263168.0, 'CT') not fixed in all samples
Mutation ('II', 478197.0, 'C') not fixed in all samples
Mutation ('IV', 273629.0, 'GTTTTTTTTTTTTTTTTTTTTTTC') not fixed in all samples
Mutation ('IV', 392146.0, 'GACACACACACACACACACACACACACACC') not fixed in

In [9]:
#print data for alleles to check frequency and read depth
name = ('VIII', 21.0, 'ACCA')

all_alleles.get_group((name[0],int(name[1]),name[2]))


,Unnamed: 0,Chromosome,Position,Alt,Time,Ref,Is_repeat,Read_depth,Num_ref,Num_alt,Freq_ref,Freq_alt,Qual_ref,Qual_alt,Treatment,Strain
8670,8670,VIII,21,ACCA,1,ACA,0,3,0,0,0.000000,0.000000,0.0,2.000000,C,1
8671,8671,VIII,21,ACCA,3,ACA,0,12,1,4,0.200000,0.800000,2.0,28.000000,C,1
8672,8672,VIII,21,ACCA,5,ACA,0,17,0,8,0.000000,1.000000,0.0,32.500000,C,1
8673,8673,VIII,21,ACCA,7,ACA,0,10,0,4,0.000000,1.000000,0.0,35.000000,C,1
8674,8674,VIII,21,ACCA,9,ACA,0,13,0,5,0.000000,1.000000,0.0,35.200000,C,1
8675,8675,VIII,21,ACCA,12,ACA,0,7,1,0,0.142857,0.000000,0.0,35.200000,C,1
28080,28080,VIII,21,ACCA,1,ACA,0,NaN,NaN,0,NaN,0.000000,NaN,NaN,C,2
28081,28081,VIII,21,ACCA,3,ACA,0,11,0,3,0.000000,1.000000,0.0,31.666667,C,2
28082,28082,VIII,21,ACCA,5,ACA,0,NaN,NaN,0,NaN,0.000000,NaN,NaN,C,2
28083,28083,VIII,21,ACCA,7,ACA,0,7,0,3,0.000000,0.428571,0.0,32.000000,C,2


In [10]:
unique2P

[('MITO', 320, 'T'), ('XII', 944108, 'C'), ('XII', 1071409, 'C')]

In [11]:
#print parent alleles to check frequency and read depth
parent_grps = parent_df.groupby(['Chromosome','Position','Alt'])
name = ('XII', 1071409, 'C')
parent_grps.get_group((name[0],int(name[1]),name[2]))

,Chromosome,Position,Ref,Alt,Ref_count,Alt_count,Ref_freq,Alt_freq,N_calls
613,XII,1071409,A,C,0,2,0,1,2


## Conclusions
- The majority of fixed alleles in the ancestor are also found at frequencies near 1 in nearly all samples and are therefore likely fixed in both. 
- All alleles fixed in ancestor but not present in other strains are in regions of very low sequencing depth and therefore explainable as high frequency sequencing errors. 
- Alleles that were not universally present at frequencies close to 1 in the samples seem to have low coverage in certain libraries, leading to high frequency sequencing errors, or regions where coverage was zero and therefore freqeuncy calls were labeled zero. 

### In summary, there appear to be around 200 mutations differing between the ancestor and the S288C reference sequence and these are present in all experimental samples